In [1]:
from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
import pandas as pd

In [2]:
!pip install sentence-transformers prince scikit-learn
import plotly.express as px
import pandas as pd
import prince

In [3]:
from sklearn.manifold import LocallyLinearEmbedding

In [4]:
def dim_red(mat, p , method):
  if (method == "ACP"):
    df = pd.DataFrame(mat)
    pca  = prince.PCA(n_components=p)
    red_mat = pca.fit_transform(df)
    red_mat = np.array(red_mat)
    return red_mat
  elif(method=="LLE"):
    lle = LocallyLinearEmbedding(n_neighbors=10, n_components=p)
    red_mat = lle.fit_transform(mat)
    return red_mat
  else:
    raise Exception("Please select one of the three methods : APC, AFC, UMAP, LLE")

In [5]:
def clust_kmeans(mat,k):
  kmeans = KMeans(n_clusters=k,n_init=15)
  kmeans.fit(mat)
  return kmeans.predict(mat)

In [6]:
def clust_AgglomerativeClustering(mat,k):
  hierarchical = AgglomerativeClustering(n_clusters=k)
  pred = hierarchical.fit_predict(mat)
  return pred

In [7]:
def clust_DBSCAN(mat,k):
  dbscan = DBSCAN(eps=0.5, min_samples=k)
  pred = dbscan.fit_predict(mat)
  return pred

In [8]:
def clust(mat, k,method = "KMeans"):
  # Apply K-means clustering
  if (method == "KMeans"):
    return clust_kmeans(mat,k)
  elif (method == "Agglomerative"):
    return clust_AgglomerativeClustering(mat,k)
  elif (method == "DBScan"):
    return clust_DBSCAN(mat,k)
  else:
    raise Exception("Please select one of the three methods : KMeans, agglomerative, DBScan")

In [9]:
import plotly.subplots as sp
import plotly.graph_objs as go
import plotly.express as px

In [10]:
def plot_clustering_2D(df_clust):
  fig = sp.make_subplots(rows=1, cols=2, subplot_titles=['Predicted Labels', 'True Labels']  )

  # Scatter plot with predicted labels
  scatter_pred = go.Scatter(
        x=df_clust['Dimension 1'],
        y=df_clust['Dimension 2'],

      mode='markers',
      marker=dict(color=df_clust['Cluster'], size=5, opacity=0.8),
      name='Predicted Labels'
  )
  fig.add_trace(scatter_pred, row=1, col=1)

  # Scatter plot with true labels
  scatter_true = go.Scatter(
        x=df_clust['Dimension 1'],
        y=df_clust['Dimension 2'],
      mode='markers',
      marker=dict(color=df_clust['Label'], size=5, opacity=0.8),
      name='True Labels'
  )
  fig.add_trace(scatter_true, row=1, col=2)

  # Update layout
  fig.update_layout(showlegend=False)

  # Show the plot
  fig.show()

In [11]:
from plotly.subplots import make_subplots

In [12]:
def plot_clustering_3D(df_clust , method  = ""):
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Predicted Labels', 'True Labels'],
                        specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]])

    # Scatter plot with predicted labels
    scatter_pred = go.Scatter3d(
        x=df_clust['Dimension 1'],
        y=df_clust['Dimension 2'],
        z=df_clust['Dimension 3'],
        mode='markers',
        marker=dict(color=df_clust['Cluster'], size=5, opacity=0.8),
        name='Predicted Labels'
    )
    fig.add_trace(scatter_pred, row=1, col=1)

    # Scatter plot with true labels
    scatter_true = go.Scatter3d(
        x=df_clust['Dimension 1'],
        y=df_clust['Dimension 2'],
        z=df_clust['Dimension 3'],
        mode='markers',
        marker=dict(color=df_clust['Label'], size=5, opacity=0.8),
        name='True Labels'
    )
    fig.add_trace(scatter_true, row=1, col=2)

    # Update layout
    fig.update_layout(showlegend=False)

    # Show the plot
    fig.show()

# Example usage
# plot_clustering_3D(your_dataframe)


In [13]:
import pickle
with open("data.pkl" , "rb") as file:
  embeddings,labels = pickle.load(file)

In [14]:
# import data
k = len(set(labels))

# embedding
method = "LLE"
# perform dimentionality reduction
red_emb = dim_red(embeddings, 20,method )
red_emb_3d_plot = dim_red(red_emb, 3 ,method )


# perform clustering
pred = clust(red_emb, k)
df_clust = pd.DataFrame({'Dimension 1': red_emb_3d_plot[:, 0], 'Dimension 2': red_emb_3d_plot[:, 1], 'Dimension 3': red_emb_3d_plot[:, 2], 'Cluster': pred, 'Label': labels})
plot_clustering_2D(df_clust)
plot_clustering_3D(df_clust)

# evaluate clustering results
nmi_score = normalized_mutual_info_score(pred,labels)
ari_score = adjusted_rand_score(pred,labels)

print(f'NMI: {nmi_score:.2f} \nARI: {ari_score:.2f}')


NMI: 0.33 
ARI: 0.08


In [15]:
import warnings
# Fit clustering 20 times
nmi_scores = []
ari_scores = []
n_iterations = 20
# Suppress FutureWarning related to KMeans
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)
for _ in range(n_iterations):
  pred = clust(red_emb, k)
  df_clust = pd.DataFrame({'Dimension 1': red_emb_3d_plot[:, 0], 'Dimension 2': red_emb_3d_plot[:, 1], 'Dimension 3': red_emb_3d_plot[:, 2], 'Cluster': pred, 'Label': labels})
  # Evaluate clustering results
  nmi_score = normalized_mutual_info_score(pred, labels)
  ari_score = adjusted_rand_score(pred, labels)

  # Append scores for mean calculation
  nmi_scores.append(nmi_score)
  ari_scores.append(ari_score)

# Compute mean scores
mean_nmi = np.mean(nmi_scores)
mean_ari = np.mean(ari_scores)

# Print mean scores
print(f'Mean NMI: {mean_nmi:.2f}')
print(f'Mean ARI: {mean_ari:.2f}')


Mean NMI: 0.31
Mean ARI: 0.07


In [16]:
iterations = [i for i in range(n_iterations)]
# Create subplots
fig = sp.make_subplots(rows=2, cols=1, subplot_titles=[f'NMI Scores Mean: {np.mean(nmi_scores):.2f} +/- {np.std(nmi_scores):.2f}',
        f'ARI Scores Mean: {np.mean(ari_scores):.2f} +/- {np.std(ari_scores):.2f}'], shared_xaxes=True)

# Add traces to the subplots

fig.add_trace(go.Scatter(x=iterations, y=nmi_scores, mode='lines+markers', name='NMI Scores'), row=1, col=1)
fig.add_trace(go.Scatter(x=iterations, y=ari_scores, mode='lines+markers', name='ARI Scores'), row=2, col=1)

# Update layout
fig.update_layout(title_text='NMI and ARI Scores Over Iterations', showlegend=True)

# Show the plot
fig.show()
